# Vectorized String Operations

Una fortaleza de Python es su relativa facilidad para manejar y manipular datos de cadenas.

Pandas se basa en esto y proporciona un conjunto completo de *operaciones de cadenas vectorizadas* que se convierten en una pieza esencial del tipo de manipulación necesaria cuando se trabaja con (léase: limpieza) datos del mundo real.

En esta sección, analizaremos algunas de las operaciones de cadenas de Pandas y luego veremos cómo usarlas para limpiar parcialmente un conjunto de datos muy desordenado de recetas recopiladas de Internet.

In [6]:
data = ['peter', 'Paul', None, 'MARY', 'gUIDO']

Pandas incluye características para abordar tanto esta necesidad de operaciones de cadenas vectorizadas como para manejar correctamente los datos faltantes a través del atributo **``str`` de los objetos de índice y serie de Pandas** que contienen cadenas.

Entonces, por ejemplo, supongamos que creamos una serie Pandas con estos datos:

In [7]:
import pandas as pd
names = pd.Series(data)
names

0    peter
1     Paul
2     None
3     MARY
4    gUIDO
dtype: object

Ahora podemos llamar a un método único que pondrá en mayúscula todas las entradas, y omitirá los valores faltantes:

In [8]:
names.str.capitalize()

0    Peter
1     Paul
2     None
3     Mary
4    Guido
dtype: object

Utiliza apply resultará en un fallo cuando aparezcan missing values

In [9]:
names.apply(lambda x: x.capitalize())

AttributeError: 'NoneType' object has no attribute 'capitalize'

## Tablas de métodos para los tipo Strings:

In [10]:
monte = pd.Series(['Graham Chapman', 'John Cleese', 'Terry Gilliam',
                   'Eric Idle', 'Terry Jones', 'Michael Palin', 'Aaron Bach'])

### Métodos similares a los métodos para String

Casi todos los métodos de String integrados de Python se reflejan en un método de String vectorizado de Pandas. Aquí hay una lista de métodos ``str`` de Pandas que reflejan los métodos de cadStringena de Python:

|             |                  |                  |                  |
|-------------|------------------|------------------|------------------|
|``len()``    | ``lower()``      | ``translate()``  | ``islower()``    | 
|``ljust()``  | ``upper()``      | ``startswith()`` | ``isupper()``    | 
|``rjust()``  | ``find()``       | ``endswith()``   | ``isnumeric()``  | 
|``center()`` | ``rfind()``      | ``isalnum()``    | ``isdecimal()``  | 
|``zfill()``  | ``index()``      | ``isalpha()``    | ``split()``      | 
|``strip()``  | ``rindex()``     | ``isdigit()``    | ``rsplit()``     | 
|``rstrip()`` | ``capitalize()`` | ``isspace()``    | ``partition()``  | 
|``lstrip()`` |  ``swapcase()``  |  ``istitle()``   | ``rpartition()`` |


In [ ]:
# Algunos devuelven strings modificados
monte.str.lower()

0    graham chapman
1       john cleese
2     terry gilliam
3         eric idle
4       terry jones
5     michael palin
6        aaron bach
dtype: object

In [ ]:
# Otros devuelven números
monte.str.len()

0    14
1    11
2    13
3     9
4    11
5    13
6    10
dtype: int64

In [11]:
# O booleanos
monte[monte.str.startswith('t')]

Series([], dtype: object)

In [ ]:
# Incluso listas
monte.str.split()

0    [Graham, Chapman]
1       [John, Cleese]
2     [Terry, Gilliam]
3         [Eric, Idle]
4       [Terry, Jones]
5     [Michael, Palin]
dtype: object

### También podemos usar regex!


| Method | Description |
|--------|-------------|
| ``match()`` | Call ``re.match()`` on each element, returning a boolean. |
| ``extract()`` | Call ``re.match()`` on each element, returning matched groups as strings.|
| ``findall()`` | Call ``re.findall()`` on each element |
| ``replace()`` | Replace occurrences of pattern with some other string|
| ``contains()`` | Call ``re.search()`` on each element, returning a boolean |
| ``count()`` | Count occurrences of pattern|
| ``split()``   | Equivalent to ``str.split()``, but accepts regexps |
| ``rsplit()`` | Equivalent to ``str.rsplit()``, but accepts regexps |

In [ ]:
# Encuentra la primera palabra completa
monte.str.extract('([A-Za-z]+)', expand=False)

0     Graham
1       John
2      Terry
3       Eric
4      Terry
5    Michael
6      Aaron
dtype: object

In [ ]:
monte.str.extract('([A-Za-z]+)', expand=True)

,0
0,Graham
1,John
2,Terry
3,Eric
4,Terry
5,Michael


In [ ]:
monte

0    Graham Chapman
1       John Cleese
2     Terry Gilliam
3         Eric Idle
4       Terry Jones
5     Michael Palin
6        Aaron Bach
dtype: object

In [ ]:
# Encuentra los nombre y apellidos que no empiezan por vocal
monte.str.findall(r'^[^AEIOU].*[^aeiou]$')

0    [Graham Chapman]
1                  []
2     [Terry Gilliam]
3                  []
4       [Terry Jones]
5     [Michael Palin]
6                  []
dtype: object

The ability to concisely apply regular expressions across ``Series`` or ``Dataframe`` entries opens up many possibilities for analysis and cleaning of data.

### Métodos de funcionalidad variada

| Method | Description |
|--------|-------------|
| ``get()`` | Index each element |
| ``slice()`` | Slice each element|
| ``slice_replace()`` | Replace slice in each element with passed value|
| ``cat()``      | Concatenate strings|
| ``repeat()`` | Repeat values |
| ``normalize()`` | Return Unicode form of string |
| ``pad()`` | Add whitespace to left, right, or both sides of strings|
| ``wrap()`` | Split long strings into lines with length less than a given width|
| ``join()`` | Join strings in each element of the Series with passed separator|
| ``get_dummies()`` | extract dummy variables as a dataframe |

#### Slicing


In [ ]:
# Similar al que vimos con variables en el ramp up
monte.str[0:3]

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
6    Aar
dtype: object

In [ ]:
# Y se puede hacer nesting de métodos, ejecutándose de izquierda a derecha
# 1. Parte las cadenas por los espacios y devuelve una lista con tantos elementos como se haya troceado la original
# 2. Devuelve el último elemento de la lista generada en el punto 1.
monte.str.split().str.get(-1)

0    Chapman
1     Cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
6       Bach
dtype: object

#### Indicator

Otro método que requiere un poco de explicación adicional es el método ``get_dummies()``.

Esto es útil cuando sus datos tienen una columna que contiene algún tipo de indicador codificado (en ML se utilizan mucho para convertir variables cualitativas en cuantitativas).

Por ejemplo, podríamos tener un conjunto de datos que contenga información en forma de códigos, como

A="nacido en Estados Unidos"
B="nacido en el Reino Unido"
C="le gusta el queso"
D="le gusta el spam"

In [ ]:
full_monte = pd.DataFrame({'name': monte,
                           'info': ['B,C,D', 'B,D', 'A,C',
                                    'B,D', 'B,C', 'B,C,D', 'A']})
full_monte

,name,info
0,Graham Chapman,"B,C,D"
1,John Cleese,"B,D"
2,Terry Gilliam,"A,C"
3,Eric Idle,"B,D"
4,Terry Jones,"B,C"
5,Michael Palin,"B,C,D"
6,Aaron Bach,A


In [ ]:
# Genera una columna para cada letra distinta en info
# Fila por fila, rellena con 0 si esa letra no estaba presente y 1 en caso contrario
full_monte['info'].str.get_dummies(',')

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1
6,1,0,0,0


In [ ]:
pin = pd.read_csv("penguins_size.csv")

In [ ]:
pin.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [ ]:
pin['island'].str.get_dummies(',')

,Biscoe,Dream,Torgersen
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
339,1,0,0
340,1,0,0
341,1,0,0
342,1,0,0


In [ ]:
pd.get_dummies(pin[['island', 'sex']])

,island_Biscoe,island_Dream,island_Torgersen,sex_.,sex_FEMALE,sex_MALE
0,False,False,True,False,False,True
1,False,False,True,False,True,False
2,False,False,True,False,True,False
3,False,False,True,False,False,False
4,False,False,True,False,True,False
...,...,...,...,...,...,...
339,True,False,False,False,False,False
340,True,False,False,False,True,False
341,True,False,False,False,False,True
342,True,False,False,False,True,False


In [ ]:
pin['sex'].unique()

array(['MALE', 'FEMALE', nan, '.'], dtype=object)

In [ ]:
pin[['island', 'sex']].str.get_dummies(',')

AttributeError: 'DataFrame' object has no attribute 'str'